In [1]:
%%time
from lhotse.kaldi import load_kaldi_data_dir
from neural_sp.datasets.asr import build_dataloader
from lhotse import CutSet
from lhotse import Fbank as feature_pipeline
from lhotse.features.io import LilcomFilesWriter, NumpyFilesWriter
from concurrent.futures import ProcessPoolExecutor
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
import os
import kaldiio

/home/yyu/Repos/neural_sp_yang/tools/miniconda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


CPU times: user 798 ms, sys: 169 ms, total: 967 ms
Wall time: 1.06 s


In [2]:
def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size / 1024 / 1024

In [17]:
%%time
! ./run.sh

                                   CSJ                                    
                    Feature extranction (stage:1)                          
steps/make_fbank.sh --nj 32 --cmd run.pl --mem 2G --write_utt2num_frames true /home/yyu/Repos/neural_sp_yang/examples/csj/s5/data_ori/train_all /home/yyu/Repos/neural_sp_yang/examples/csj/s5/data_ori/log/make_fbank/train_all /home/yyu/Repos/neural_sp_yang/examples/csj/s5/data_ori/fbank
utils/validate_data_dir.sh: Successfully validated data-directory /home/yyu/Repos/neural_sp_yang/examples/csj/s5/data_ori/train_all
steps/make_fbank.sh [info]: segments file exists: using that.
steps/make_fbank.sh: It seems not all of the feature files were successfully procesed (417738 != 417763); consider using utils/fix_data_dir.sh /home/yyu/Repos/neural_sp_yang/examples/csj/s5/data_ori/train_all
steps/make_fbank.sh: Succeeded creating filterbank features for train_all
steps/make_fbank.sh --nj 32 --cmd run.pl --mem 2G --write_utt2num_frames true /home/y

In [3]:
cd /home/yyu/Repos/neural_sp_yang/examples/csj/s5/

/home/yyu/Repos/neural_sp_yang/examples/csj/s5


In [4]:
## Preparation for eval1

## CutSet
recording_set, supervision_set= load_kaldi_data_dir("data/eval1", 16000)
cut_set_numpy = CutSet.from_manifests(recordings=recording_set, supervisions=supervision_set)
cut_set_llc = CutSet.from_manifests(recordings=recording_set, supervisions=supervision_set)

feature_extractor_numpy = feature_pipeline()
feature_extractor_llc = feature_pipeline()

ex = ProcessPoolExecutor(10)
    
cut_set_numpy = cut_set_numpy.compute_and_store_features(
                extractor=feature_extractor_numpy,
                storage_path="temp/numpy",
                storage_type=NumpyFilesWriter,
                executor=ex,
                num_jobs=10)
cut_set_numpy = cut_set_numpy.trim_to_supervisions()
cut_set_llc = cut_set_llc.compute_and_store_features(
                extractor=feature_extractor_llc,
                storage_path="temp/llc",
                storage_type=LilcomFilesWriter,
                executor=ex,
                num_jobs=10)
cut_set_llc = cut_set_llc.trim_to_supervisions()

Extracting and storing features (chunks progress):   0%|          | 0/10 [00:00<?, ?it/s]

/home/yyu/Repos/neural_sp_yang/tools/miniconda/lib/python3.7/site-packages/torchaudio/compliance/kaldi.py:574: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  fft = torch.rfft(strided_input, 1, normalized=False, onesided=True)
/home/yyu/Repos/neural_sp_yang/tools/miniconda/lib/python3.7/site-packages/torchaudio/compliance/kaldi.py:574: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  fft = torch.rfft(strided_input, 1, normalized=False, onesided=True)
/home/yyu/Repos/neural_sp_yang/tools/miniconda/l

Extracting and storing features (chunks progress):   0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
cut_set_numpy.to_yaml("eval1.numpy.cut.yaml")
cut_set_llc.to_yaml("eval1.llc.cut.yaml")

In [14]:
!python -W ignore /home/yyu/Repos/neural_sp_yang//utils/cutset_to_tsv.py \
                        --unit wp \
                        --dict data/dict/train_nodev_all_wpbpe10000.txt \
                        --remove_space false \
                        --unk "<unk>" \
                        --space "<space>" \
                        --nlsyms "" \
                        --wp_model data/dict/train_nodev_all_bpe10000 \
                        --wp_nbest 1 \
                        --jobs 10 \
                        --cutset_yaml eval1.numpy.cut.yaml \
                        --tsv_location eval1.numpy.cut.tsv

/bin/bash: space: No such file or directory


In [ ]:
!python -W ignore /home/yyu/Repos/neural_sp_yang//utils/cutset_to_tsv.py \
                        --unit wp \
                        --dict data/dict/train_nodev_all_wpbpe10000.txt \
                        --remove_space false \
                        --unk "<unk>" \
                        --space "<space>" \
                        --nlsyms "" \
                        --wp_model data/dict/train_nodev_all_bpe10000 \
                        --wp_nbest 1 \
                        --jobs 10 \
                        --cutset_yaml eval1.llc.cut.yaml \
                        --tsv_location eval1.llc.cut.tsv

In [6]:
import pandas as pd

titile_list = ['utt_id', 'speaker', 'feat_path', 'xlen', 'xdim', 
                       'text', 'token_id', 'ylen', 'ydim', 'offset', 'cut_id']

df_llc = pd.read_csv("eval1.llc.cut.tsv", encoding='utf-8', delimiter='\t')
df_llc = df_llc.loc[:, titile_list]

df_numpy = pd.read_csv("eval1.numpy.cut.tsv", encoding='utf-8', delimiter='\t')
df_numpy = df_numpy.loc[:, titile_list]

In [7]:
def collate_features_lhotse(cuts: CutSet):
    """
    Load features for all the cuts and return them as a batch in a torch tensor.
    The output shape is ``(batch, time, features)``.
    The cuts will be padded with silence if necessary.
    """
    assert all(cut.has_features for cut in cuts)
    first_cut = next(iter(cuts))
    #features = torch.empty(len(cuts), first_cut.num_frames, first_cut.num_features)
    features = []
    for idx, cut in enumerate(cuts):
        features.append(cut.load_features())
    return features

In [8]:
%%time
Load_from_numpy_lhotse = collate_features_lhotse(cut_set_numpy)
print("%i features are collacted"%(len(Load_from_numpy_lhotse)))
print("Size to store lhotse features in numpy: %.2f MB"%(get_size("temp/numpy")))

1272 features are collacted
Size to store lhotse features in numpy: 379.73 MB
CPU times: user 417 ms, sys: 5.58 s, total: 6 s
Wall time: 6 s


In [9]:
%%time
Load_from_llc_lhotse = collate_features_lhotse(cut_set_llc)
print("%i features are collacted"%(len(Load_from_llc_lhotse)))
print("Size to store lhotse features in Lilcom: %.2f MB"%(get_size("temp/llc")))

1272 features are collacted
Size to store lhotse features in Lilcom: 108.10 MB
CPU times: user 53.5 s, sys: 6.3 s, total: 59.8 s
Wall time: 59.8 s


In [10]:
def collate_features_lhotse_neuralsp(df, storage_type):
    """
    Load features for all the cuts and return them as a batch in a torch tensor.
    The output shape is ``(batch, time, features)``.
    The cuts will be padded with silence if necessary.
    """

    indices = range(len(df))
    features = []
    if storage_type == "Lilcom":
        from lhotse import LilcomFilesReader as FilesReader
    elif storage_type == "Numpy":
        from lhotse import NumpyFilesReader as FilesReader

    for i in indices:
        storage_path, storage_key = df['feat_path'][i].split("@")
        left_offset_frames = df['offset'][i]
        right_offset_frames = left_offset_frames + df['xlen'][i]
        FeatureFiles = FilesReader(storage_path)
        features.append(FeatureFiles.read(storage_key, left_offset_frames, right_offset_frames))

    return features

In [11]:
%%time
Load_from_llc_lhotse_neuralsp = collate_features_lhotse_neuralsp(df_llc, "Lilcom")
print("%i features are collacted"%(len(Load_from_llc_lhotse_neuralsp)))
print("Size to store lhotse features in Lilcom: %.2f MB"%(get_size("temp/llc")))

1272 features are collacted
Size to store lhotse features in Lilcom: 108.10 MB
CPU times: user 53.6 s, sys: 6.45 s, total: 1min
Wall time: 1min


In [12]:
%%time
Load_from_numpy_lhotse_neuralsp = collate_features_lhotse_neuralsp(df_numpy, "Numpy")
print("%i features are collacted"%(len(Load_from_numpy_lhotse_neuralsp)))
print("Size to store lhotse features in Lilcom: %.2f MB"%(get_size("temp/llc")))

1272 features are collacted
Size to store lhotse features in Lilcom: 108.10 MB
CPU times: user 594 ms, sys: 17.6 s, total: 18.2 s
Wall time: 18.2 s


In [16]:
def collate_features_kaldiio(df):
    features = []
    indices = range(len(df))
    features = [kaldiio.load_mat(df['feat_path'][i]) for i in indices]
    return features

In [17]:
import pandas as pd

titile_list = ['utt_id', 'speaker', 'feat_path', 'xlen', 'xdim', 
                           'text', 'token_id', 'ylen', 'ydim']

df_kaldi = pd.read_csv("data_ori/dataset/eval1_sp_all_wpbpe10000.tsv", encoding='utf-8', delimiter='\t')
df_kaldi = df_kaldi.loc[:, titile_list]


In [18]:
%%time
Load_from_kaldiio = collate_features_kaldiio(df_kaldi)
print("%i features are collacted"%(len(Load_from_kaldiio)))
print("Size to store lhotse features in Lilcom: %.2f MB"%(get_size("data_ori/dump/eval1_sp_all")))

1272 features are collacted
Size to store lhotse features in Lilcom: 51.34 MB
CPU times: user 681 ms, sys: 104 ms, total: 785 ms
Wall time: 790 ms
